In [ ]:
pip install nltk scikit-learn streamlit

In [ ]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression



In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
nltk.data.path.append(os.path.abspath("nltk_data"))
nltk.download('punkt')

In [ ]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up"],
        "responses": ["Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responises": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
       "tag": "age",
       "patterns": ["How old are you", "What's your age", "What is your age?", "Are you old?", "When were you created?"],  # Added more patterns
       "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
   },
    {
        "tag": "location",
        "patterns": ["Where are you located", "What is your location", "Where do you live"],
        "responses": ["I exist in the digital realm.", "I don't have a physical location.", "I am everywhere and nowhere."]
    },
    {
        "tag": "favorite_food",
        "patterns": ["What is your favorite food", "Do you like to eat", "What food do you enjoy"],
        "responses": ["As a chatbot, I don't eat.", "I don't have taste buds, so I don't have favorite foods.", "I consume data, not food."]
    },
    {
        "tag": "favorite_color",
        "patterns": ["What is your favorite color", "Do you have a favorite color", "What color do you like"],
        "responses": ["I don't perceive colors.", "I don't have a favorite color.", "I'm a digital entity, so colors are not a factor for me."]
    },
    {
        "tag": "recommendation",
        "patterns": ["Can you recommend something", "Suggest a movie", "Give me a book recommendation"],
        "responses": ["I can provide recommendations based on your interests. Please tell me what you are looking for.", "To give you a good suggestion, I need to know more about your preferences."]
    },
    {
        "tag": "joke",
        "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes"],
        "responses": ["Why don't scientists trust atoms? Because they make up everything!", "What do you call a lazy kangaroo? Pouch potato!", "Why did the bicycle fall over? Because it was two tired!"]
    },
    {
        "tag": "news",
        "patterns": ["What's the latest news", "Tell me the news", "Any news today"],
        "responses": ["I don't have access to real-time news. You can check a news website or app for the latest updates."]
    },
    {
        "tag": "time",
        "patterns": ["What time is it", "Tell me the time", "Do you know the time"],
        "responses": ["I cannot provide the current time. Please check your device's time."]
    },
    {
        "tag": "definition",
        "patterns": ["What does [word] mean", "Define [word]", "What is the definition of [word]"],
        "responses": ["Please provide the word you would like me to define."]
    },
    {
        "tag": "calculator",
        "patterns": ["Calculate [expression]", "What is [expression]", "Solve [expression]"],
        "responses": ["Please provide the mathematical expression you would like me to calculate."]
    },
    {
        "tag": "sports",
        "patterns": ["What are the latest sports scores", "Any sports news", "Tell me about sports"],
        "responses": ["I don't have access to real-time sports scores. You can check a sports website or app for the latest updates."]
    },
    {
        "tag": "travel",
        "patterns": ["Where should I travel", "Suggest a vacation spot", "Tell me about travel destinations"],
        "responses": ["To give you a good travel suggestion, I need to know more about your interests and preferences."]
    },
    {
        "tag": "shopping",
        "patterns": ["Where can I buy [item]", "Recommend a place to shop for [item]", "Where to find [item]"],
        "responses": ["I can provide suggestions based on the item you are looking for. Please specify the item."]
    },
    {
        "tag": "programming",
        "patterns": ["How do I code [language]", "What is [programming concept]", "Explain [programming topic]"],
        "responses": ["Please provide the programming language or concept you would like me to explain."]
    },
    {
        "tag": "health",
        "patterns": ["What are healthy habits", "How to stay healthy", "Tips for better health"],
        "responses": ["Maintaining a balanced diet, regular exercise, adequate sleep, and stress management are key to staying healthy. Consult a healthcare professional for personalized advice."]
    },
    {
        "tag": "music",
        "patterns": ["Recommend a song", "What are popular songs", "Tell me about music"],
        "responses": ["To give you a good music suggestion, I need to know more about your music preferences."]
    },
    {
        "tag": "movies",
        "patterns": ["Recommend a movie", "What are good movies", "Tell me about movies"],
        "responses": ["To give you a good movie suggestion, I need to know more about your movie preferences."]
    },
    {
        "tag": "books",
        "patterns": ["Recommend a book", "What are popular books", "Tell me about books"],
        "responses": ["To give you a good book suggestion, I need to know more about your reading preferences."]
    },
    {
        "tag": "languages",
        "patterns": ["What languages do you speak", "Can you speak [language]", "Tell me about languages"],
        "responses": ["I can process and generate text in many languages.", "I can understand and respond in multiple languages."]
    }

]


In [ ]:
vectorizer = TfidfVectorizer()
clf = LogisticRegression(random_state=0, max_iter=10000)

#Preprocess the data
tags = []
patterns = []
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)

In [ ]:
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x, y)

In [ ]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])
    tag = clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])
            return response

In [ ]:
user_input = "What's your age?"
response = chatbot(user_input)
print(response)

In [ ]:
user_input = "Hello"
response = chatbot(user_input)
print(response)

In [ ]:
user_input = "budget"
response = chatbot(user_input)
print(response)

In [ ]:
user_input = "hi"
response = chatbot(user_input)
print(response)

In [ ]:
user_input = "age"
response = chatbot(user_input)
print(response)

In [ ]:
user_input = "what is a credit score?"
response = chatbot(user_input)
print(response)